In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Only execute this unless more data with different countries (which aren't already fixes in the mapping) is added
def gen_missing_names_list():
    
    # read in the country names column from each of the initial datasets used in affetcable emissions
    tong_ctry = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/tong_commited_emit_data_weighted/tong_data_by_country.csv', encoding = 'latin1')['Unnamed: 0']
    scen_ctry = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/PMSSPBIE_05Feb20.csv', encoding = 'latin1')['country']
    coal_ctry = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/coal_data/january2022_coal_data/Units-Table 1.csv')['Country']
    gas_ctry = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/gas_data/january2022_gas_data/Gas Units-Table 1.csv')['Country']
    steel_ctry = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/steel_data/Global Steel Plant Tracker - February 2021 - Standard Copy.csv', dtype = 'str')['Country']
    # combine all these country names into a seires, and remove the repeats
    countries_in_data = pd.concat([tong_ctry] + [scen_ctry] + [coal_ctry] + [gas_ctry] + [steel_ctry])
    strip_dups = countries_in_data.drop_duplicates(keep='first')

    # import a two column file with correct country names in one col, and things that each country could possibly
    # be specified as ("alternative names") in the next column. Some alt names from the data aren't in this file.
    alt_names_file = pd.read_csv('/Users/Violet/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/country_mapping/countrynames.csv', index_col = 0, dtype = str, encoding = 'latin1')
    
    # output all of the country names in our data which are not found based on the matchings above, 
    # output this to a file, then, open the file locally to see what remain unspecified, and fix this
    need_to_add = []
    for alt_name in strip_dups:
        if alt_name not in alt_names_file['country.name.alt']:
            need_to_add.append(alt_name)

    adds = pd.DataFrame(need_to_add)
    adds.to_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/country_mapping/missing_alt_names.csv')
    
gen_missing_names_list()

In [ ]:
def export_cntry_data_fix():
    
    # import manually cleaned version of the file exported above:
    clean_country_mapping = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/country_mapping/countrynames_modified.csv',
               usecols = ['country.name.en', 'country.name.alt'])

    # remove any duplicate lines which might have accidentally been introduced to the file
    strip_dups = clean_country_mapping.drop_duplicates(keep='first')

    # try to replace the country names in each dataset with the country names suggested by the mapping
    tong_scen_country = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/tong_commited_emit_data_weighted/tong_data_by_country.csv', encoding = 'latin1')
    tong_cntry_fix = clean_country_mapping.merge(tong_scen_country, left_on = 'country.name.alt', right_on = 'Unnamed: 0', how = 'inner').drop(columns = ['country.name.alt', 'Unnamed: 0'])

    scenarios = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/PMSSPBIE_05Feb20.csv', encoding = 'latin1')
    scen_cntry_fix = clean_country_mapping.merge(scenarios, left_on = 'country.name.alt', right_on = 'country', how = 'inner').drop(columns = ['country.name.alt', 'country'])

    GEM_coal = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/coal_data/january2022_coal_data/Units-Table 1.csv')
    coal_cntry_fix = clean_country_mapping.merge(GEM_coal, left_on = 'country.name.alt', right_on = 'Country', how = 'inner').drop(columns = ['country.name.alt', 'Country'])

    GEM_gas = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/gas_data/january2022_gas_data/Gas Units-Table 1.csv')
    gas_cntry_fix = clean_country_mapping.merge(GEM_gas, left_on = 'country.name.alt', right_on = 'Country', how = 'inner').drop(columns = ['country.name.alt', 'Country'])

    GEM_steel = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/steel_data/Global Steel Plant Tracker - February 2021 - Standard Copy.csv', dtype = 'str')
    steel_cntry_fix = clean_country_mapping.merge(GEM_steel, left_on = 'country.name.alt', right_on = 'Country', how = 'inner').drop(columns = ['country.name.alt', 'Country'])
    
    GEM_steel = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/steel_data/Global Steel Plant Tracker - February 2021 - Standard Copy.csv', dtype = 'str')
    steel_cntry_fix = clean_country_mapping.merge(GEM_steel, left_on = 'country.name.alt', right_on = 'Country', how = 'inner').drop(columns = ['country.name.alt', 'Country'])

    tong_cntry_fix.to_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/inputs_w_country_fix/tong_cntry_fix.csv')
    scen_cntry_fix.to_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/inputs_w_country_fix/scen_cntry_fix.csv')
    coal_cntry_fix.to_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/inputs_w_country_fix/coal_cntry_fix.csv')
    gas_cntry_fix.to_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/inputs_w_country_fix/gas_cntry_fix.csv')
    steel_cntry_fix.to_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/inputs_w_country_fix/steel_cntry_fix.csv')
    
export_cntry_data_fix()